In [1]:
#Note: Can add synthesis params (like use_dsp48) as 3rd series params

In [2]:
#HRR_mode: noHR only (1), noHR or SoE_HR (2), SoE_HR only (3), SI (4)
#N start: 1, 1, 2, 2
#1: no HRR and NOEPS, 2 and 3: NOEPS given, HRR calculated, 4: HRR given and corrected if needed, NOEPS calculated

In [3]:
import sys
import math
import subprocess
import shutil
#import os
#import time

import import_ipynb
from REGISTERED_Generator import REGISTERED_Gen

from MatrixMatrixMultiply_SI_V1_2 import MatrixMatrixMultiply_SI
# from MatrixMatrixMultiply_SoE_V1_1 import MatrixMatrixMultiply_SoE

# from MatrixVectorMultiply_SoE_V2_5 import MatrixVectorMultiply_SoE
# from DotProduct_SoE_V4_5 import DotProduct_SoE

# from SItoSoE_V1_10 import SItoSoE

from MatrixMatrixMultiply_SI_testbench_F_v1_0 import MatrixMatrixMultiply_SI_testbench_F

importing Jupyter notebook from REGISTERED_Generator.ipynb
importing Jupyter notebook from MatrixMatrixMultiply_SI_V1_2.ipynb
importing Jupyter notebook from SItoSoE_V1_10.ipynb
importing Jupyter notebook from MatrixMatrixMultiply_SoE_V1_1.ipynb
importing Jupyter notebook from MatrixVectorMultiply_SoE_V2_5.ipynb
importing Jupyter notebook from DotProduct_SoE_V4_5.ipynb
importing Jupyter notebook from DotProduct_Systolic_V1_11.ipynb
importing Jupyter notebook from DotProduct_noHR_M_V3_7.ipynb
importing Jupyter notebook from RegisteredNto1Adder_V1_1.ipynb
importing Jupyter notebook from DotProduct_noHR_C_V3_7.ipynb
importing Jupyter notebook from AdderTree_ASm_V1_20.ipynb
importing Jupyter notebook from MatrixMatrixMultiply_SI_testbench_F_v1_0.ipynb


In [92]:
# # re_import after changes
# del sys.modules['MatrixMatrixMultiply_SI_testbench_F_v1_0']
# from MatrixMatrixMultiply_SI_testbench_F_v1_0 import MatrixMatrixMultiply_SI_testbench_F

In [93]:
# def CEIL(a,b):
#     return (a+b-1)//b

In [94]:
#read-only globals: NoOfParameters, HRRmode, Print_To_File, Files_Location
#writed globals: parameters_result
def generate_params(i):
    global NoOfRresults, NOEPS
    if i==NoOfParameters:
        NoOfRresults += 1
        ListOfAllParametersResults.append(parameters_result.copy())
        return
    
#     print(i,parameters[i])
    param = parameters[i]
    vals = parameter_values[param]
    start = vals[0]
    end = vals[1]
    if param == "N":
        for val in range(start,end+1):
            parameters_result[param] = val
            generate_params(i+1)
    elif param in ["M","Q"]:
        for val in range(start,end+1):
            parameters_result[param] = val
            generate_params(i+1)
    elif param == "PR":
        for val in range(start,min(end,parameters_result['M'])+1):
            parameters_result["PR"] = val
            generate_params(i+1)
    elif param == "PC":
        for val in range(start,min(end,parameters_result['Q'])+1):
            parameters_result["PC"] = val
            generate_params(i+1)
    elif param == "HRR":
        for val in range(start,min(end,parameters_result['N'])+1):
            NOEPS_check = math.ceil(parameters_result["N"]/val)
            if math.ceil(parameters_result["N"]/NOEPS_check) == val:
                parameters_result["HRR"] = val
                generate_params(i+1)
            #else HRR is not valid
    elif param == "NOEPS":
        if HRRmode == 2:
            endH = parameters_result["N"]
        elif HRRmode == 3:
            endH = parameters_result["N"]-1
        for val in range(start,min(end,endH)+1):
                parameters_result["NOEPS"] = val
                generate_params(i+1)
    elif param == "MAMCS":
        if HRRmode == 1:
            NOEPS = parameters_result["N"]
        elif HRRmode in [2,3]:
            NOEPS = parameters_result["NOEPS"]
        elif HRRmode == 4:
            NOEPS = math.ceil(parameters_result["N"]/parameters_result["HRR"])
        for val in range(start,min(end,NOEPS)+1):
            parameters_result["MAMCS"] = val
            generate_params(i+1)
    elif param == "ADDER_SIZE":
        if parameters_result["MAMCS"] == NOEPS:
            parameters_result["ADDER_SIZE"] = 0
            generate_params(i+1)
        else: #MAMCS < NOEPS
            for val in range(start,min(end,math.ceil(NOEPS/parameters_result["MAMCS"]))+1):
                parameters_result["ADDER_SIZE"] = val
                generate_params(i+1)
    elif param == "ENABLE_COLUMN_LATCH":
        if math.ceil(parameters_result["M"]/parameters_result["PR"]) == 1:
            parameters_result["ENABLE_COLUMN_LATCH"] = 0
            generate_params(i+1)
        else:
            for val in range(start,end+1):
                parameters_result["ENABLE_COLUMN_LATCH"] = val
                generate_params(i+1)
    elif param == "ENABLE_ROW_LATCH":
        if math.ceil(parameters_result["Q"]/parameters_result["PC"]) == 1:
            parameters_result["ENABLE_ROW_LATCH"] = 0
            generate_params(i+1)
        else:
            for val in range(start,end+1):
                parameters_result["ENABLE_ROW_LATCH"] = val
                generate_params(i+1)
    elif param in ["ENABLE_INPUT_LATCH","ENABLE_SIM_OUTPUT_REGISTERS"]:
        for val in range(start,end+1):
            parameters_result[param] = val
            generate_params(i+1)

In [95]:
def work_on_random_results_cpu():
    global NoOfResultsT, NoOfCPUResults, parameters_result_current
    
    CPU_Counter = 0
    
    for index in RandomIndices:
        if CPU_Counter==this_CPU:
            NoOfCPUResults += 1
            
            parameters_result_current = ListOfAllParametersResults[index]

            if print_result_flag:
                print("{}:".format(index))
                print(parameters_result_current)
            
            if make_base_files_flag:
                make_files()
                NoOfResultsT += NoOfVResults
            
            if print_result_flag or make_base_files_flag:
                print()

        if CPU_Counter==(CPU_Cores_Count-1):
            CPU_Counter=0
        else:
            CPU_Counter+=1

In [96]:
def make_files():
    global top, Current_Files_Location, NoOfVResults, modules
    modules = {}
    top = function(parameters_result_current, Print_To_File, Files_Location, 1, modules)
    print(modules)
    print(top)
    Current_Files_Location = Files_Location+top+"/"
    REGISTERED_Gen(top, Print_To_File, Current_Files_Location)
    make_prj()
    make_tcl()
    NoOfVResults = 0
    generate_verilog_params(0)
    print("no of verilog param modes = {}".format(NoOfVResults))

In [97]:
def make_prj():
    with open(Current_Files_Location+top+"_REGISTERED.prj","w") as prj_file:
        print("verilog work \""+top+"_REGISTERED.v\"", file=prj_file)
        for module in modules:
            print("verilog work \""+module+".v\"", file=prj_file)

In [98]:
def make_tcl():
    if Print_To_File<=0:
        tcl_file=sys.stdout
    else:
        tcl_file=open(Current_Files_Location+top+".tcl", 'w+')
        
    print("set myProject \""+top+"\"\n", file=tcl_file)
    
    print("proc set_project_props {} {", file=tcl_file)
    print("\tproject set family \""+family+"\"", file=tcl_file)
    print("\tproject set device \""+device+"\"", file=tcl_file)
    print("\tproject set package \""+package+"\"", file=tcl_file)
    print("\tproject set speed \""+speed_grade+"\"\n}\n", file=tcl_file)

    print("proc add_source_files {} {", file=tcl_file)
    print("\txfile add \""+top+"_REGISTERED.v\"", file=tcl_file)
    for module in modules:
        print("\txfile add \""+module+".v\"", file=tcl_file)
    print("\t#Set the Top Module", file=tcl_file)
    print("\tproject set top \""+top+"_REGISTERED\"\n}\n", file=tcl_file)
    
    print("""set proj_exts [ list ise xise gise ]
foreach ext $proj_exts {
   set proj_name "${myProject}.$ext"
   if { [ file exists $proj_name ] } { 
      file delete $proj_name
   }
}

project new $myProject
set_project_props
add_source_files
project close""", file=tcl_file)

    if Print_To_File>0:
        tcl_file.close()
    
    with open(Current_Files_Location+"CreateProject.bat","w") as CPB_file:
        print("xtclsh "+top+".tcl 2> TCLinfo.log", file=CPB_file)

Verilog Parameter Related Functions:

In [ ]:
#ListOfAllVerilogParametersResults.append(verilog_parameters_result.copy())

In [99]:
def generate_verilog_params(i):
    global NoOfVResults, IW_set
    if i==NoOfVeilogParameters:
        NoOfVResults += 1
        #print(verilog_parameters_result)
        make_verilog_params_files()
        return
    if i==0:
        IW_set = set()        
    param = verilog_parameters[i]
    vals = verilog_parameter_values[param]
    start = vals[0]
    end = vals[1]
    if param in ["IN_WIDTH","INPUT_REG_DEPTH","MULT_PIPE_DEPTH"]:
#         for val in vals:
        for val in range(start,end+1):
            verilog_parameters_result[param] = val
            generate_verilog_params(i+1)
    return NoOfVResults

In [100]:
def make_verilog_params_files():
    global VP_name_append
    VP_name_append = ""
    for param in verilog_parameters_ordered:
        if param in verilog_parameters_result:
            VP_name_append += "_{}".format(verilog_parameters_result[param])
    
    if make_Synthesize_files_flag:
        make_SYN()
        make_TMPR()
        if Synthesize_flag:
            Synthesize_Project()

    if has_testbench and make_Simulate_files_flag:
        make_Simulate()
        if Simulate_flag:
            Simulate_Project()

Synthesize Related Functions:

In [101]:
#current layout: spartan 6
def make_SYN():
    global STMPR_file_name, SynthesizeCommand
#     if Print_To_File<=0:
#         xst_file=sys.stdout
#     else:
#         STMPR_file_name = top+"_REGISTERED"+VP_name_append
#         xst_file=open(Current_Files_Location+STMPR_file_name+".xst", "w")
#     if True:

    STMPR_file_name = top+"_REGISTERED"+VP_name_append
    with open(urrent_Files_Location+STMPR_file_name+".xst", "w") as xst_file:

        print("""set -tmpdir "./"
    set -xsthdpdir "xst"
    run""", file=xst_file)
        print("-ifn "+top+"_REGISTERED.prj", file=xst_file)
        print("-ofn "+top+"_REGISTERED"+VP_name_append, file=xst_file)
    #     print("-ofn "+top+"_REGISTERED", end='', file=xst_file)
    #     for param in verilog_parameters_ordered:
    #         if param in verilog_parameters_result:
    #         print("_{}".format(verilog_parameters_result[param]), end='', file=xst_file)
    #     print(file=xst_file)
        print("-ofmt NGC", file=xst_file)
        print("-p "+device+speed_grade+"-"+package, file=xst_file)
        print("-top "+top+"_REGISTERED", file=xst_file)
        print("""-opt_mode Speed
    -opt_level 1
    -power NO
    -iuc NO
    -keep_hierarchy No
    -netlist_hierarchy As_Optimized
    -rtlview Yes
    -glob_opt AllClockNets
    -read_cores YES
    -write_timing_constraints NO
    -cross_clock_analysis NO
    -hierarchy_separator /
    -bus_delimiter <>
    -case Maintain
    -slice_utilization_ratio 100
    -bram_utilization_ratio 100
    -dsp_utilization_ratio 100
    -lc Auto
    -reduce_control_sets Auto
    -fsm_extract YES -fsm_encoding Auto
    -safe_implementation No
    -fsm_style LUT
    -ram_extract Yes
    -ram_style Auto
    -rom_extract Yes
    -shreg_extract YES
    -rom_style Auto
    -auto_bram_packing NO
    -resource_sharing YES
    -async_to_sync NO
    -shreg_min_size 2
    -use_dsp48 Auto
    -iobuf YES
    -max_fanout 100000
    -bufg 16
    -register_duplication YES
    -register_balancing No
    -optimize_primitives NO
    -use_clock_enable Auto
    -use_sync_set Auto
    -use_sync_reset Auto
    -iob Auto
    -equivalent_register_removal YES
    -slice_utilization_ratio_maxmargin 5""", file=xst_file)
        print("-generics {", end='', file=xst_file)
        for param in verilog_parameters_result:
            print(param+"={} ".format(verilog_parameters_result[param]), end='', file=xst_file)
        print("}", end='', file=xst_file)

#     if Print_To_File>0:
#         xst_file.close()
    
    with open(Current_Files_Location+"Synthesize"+VP_name_append+".bat","w") as SYNB_file:
        SynthesizeCommand = "xst -intstyle silent -ifn "+STMPR_file_name+".xst -ofn "+STMPR_file_name+".syr"
        print(SynthesizeCommand, file=SYNB_file)

In [102]:
def make_TMPR():
    with open(Current_Files_Location+"TMPR"+VP_name_append+".bat","w") as TMPRB_file:
        #print("ngdbuild -intstyle silent -quiet -dd _ngo -nt timestamp -i -p "+device+speed_grade+"-"+package+" "+STMPR_file_name+".ngc "+STMPR_file_name+".ngd", file=TMPRB_file)
        print("ngdbuild -dd _ngo -nt timestamp -i -p "+device+speed_grade+"-"+package+" "+STMPR_file_name+".ngc "+STMPR_file_name+".ngd", file=TMPRB_file)
        print("map -intstyle silent -p "+device+speed_grade+"-"+package+" -w -logic_opt off -ol high -t 1 -xt 0 -register_duplication off -r 4 -global_opt off -mt off -ir off -pr off -lc off -power off -o "+STMPR_file_name+"_map.ncd "+STMPR_file_name+".ngd "+STMPR_file_name+".pcf", file=TMPRB_file)
        print("par -w -intstyle silent -ol high -mt off "+STMPR_file_name+"_map.ncd "+STMPR_file_name+".ncd "+STMPR_file_name+".pcf", file=TMPRB_file)
        print("trce -intstyle silent -v 3 -s 3 -n 3 -fastpaths -xml "+STMPR_file_name+".twx "+STMPR_file_name+".ncd -o "+STMPR_file_name+".twr "+STMPR_file_name+".pcf", file=TMPRB_file)

In [103]:
def Synthesize_Project():
    resultS = subprocess.call(SynthesizeCommand, cwd=Current_Files_Location)
    print(VP_name_append+" Synthesize Result: ", end='')
    if resultS==0:
        print("Successful (0)")
    else:
        print("Unsuccessful ({})".format(resultS))

TestBench Related Functions:

In [104]:
#testbench is not included in tcl files list, user can add it to project manualy

In [105]:
test_glbl_location = "C:/Xilinx/14.7/ISE_DS/ISE/verilog/src/glbl.v"

In [106]:
def make_Simulate():
    global FuseCommand, SimulateCommand
    
    IWp = verilog_parameters_result["IN_WIDTH"]
    
    if IWp not in IW_set:
        IW_set.add(IWp)
        testbench_function(parameters_result_current, Print_To_File, Files_Location, verilog_parameters_result["IN_WIDTH"])
        with open(Current_Files_Location+top+"_test_IW{}_beh.prj".format(IWp),"w") as prj_file:
            #this may generate duplicate prj file (for same IWp), but no problem
            for module in modules:
                print("verilog work \""+module+".v\"", file=prj_file)
            print("verilog work \""+top+"_test_IW{}.v\"".format(IWp), file=prj_file)
            print("verilog work \""+test_glbl_location+"\"", file=prj_file)
    
    FuseCommand = "fuse -intstyle silent -nodebug -incremental -lib unisims_ver -lib unimacro_ver -o test_isim_beh"+VP_name_append+".exe -prj "+top+"_test_IW{}_beh.prj work.".format(IWp)+top+"_test_IW{} work.glbl".format(IWp)
    for param in verilog_parameters_result:
        if param!="IN_WIDTH":
            FuseCommand += " -generic_top \""+param+"={}\"".format(verilog_parameters_result[param])

    SimulateCommand = "test_isim_beh"+VP_name_append+".exe -intstyle silent -tclbatch isim.tcl -wdb test_isim_beh"+VP_name_append+".wdb"
        
    with open(Current_Files_Location+"Simulate"+VP_name_append+".bat","w") as Simulate_file:
        print(FuseCommand, file=Simulate_file)
        print(SimulateCommand, file=Simulate_file)

    shutil.copy("./isim.tcl",Current_Files_Location+"isim.tcl")

In [107]:
def Simulate_Project():
    resultFuse = subprocess.call(FuseCommand, cwd=Current_Files_Location)
    print(VP_name_append+" Fuse Result: ", end='')
    if resultFuse==0:
        print("Successful (0)")
    else:
        print("Unsuccessful ({})".format(resultFuse))
        return

    if resultFuse==0:
    #     while not os.path.isfile(Current_Files_Location+"test_isim_beh"+VP_name_append+".exe"):
    #         ;
    #     time.sleep(5)
    #     print(Current_Files_Location)
    #     print(SimulateCommandArgs)
        test_name = "test_isim_beh"+VP_name_append+".exe"
        wdb_name = "test_isim_beh"+VP_name_append+".wdb"
    #     resultSimulate = subprocess.call([test_name, "-intstyle", "silent", "-tclbatch", "isim.tcl", "-wdb", wdb_name], cwd=Current_Files_Location)
        resultSimulate = subprocess.call([test_name, "-intstyle", "silent", "-tclbatch", "isim.tcl", "-wdb", wdb_name], cwd=Current_Files_Location, shell=True)
        print(VP_name_append+" Simulate Run Return Code: ", end='')
        if resultSimulate==0:
            print("Successful (0)")
        else:
            print("Unsuccessful ({})".format(resultSimulate))
            return

        if resultSimulate==0:
            #open output file and check saimulation result
            with open(Current_Files_Location+VP_name_append+"_Simulation_Result.txt","r") as Simulation_Result_file:
                print(VP_name_append+" Simulation Result: "+Simulation_Result_file.readline(), end='')

Functions Start

In [129]:
functions_list = []
functions_names_list = []
parameters_list = []
HRR_mode_list = []
verilog_parameters_list = []
has_testbench_list = []
testbench_functions_list = []

In [130]:
#MatrixMatrixMultiply_SI

functions_list.append(MatrixMatrixMultiply_SI)
functions_names_list.append("MatrixMatrixMultiply_SI")

parameters_list.append(["M","N","Q","PR","PC","HRR","MAMCS","ADDER_SIZE",
                       "ENABLE_INPUT_LATCH","ENABLE_COLUMN_LATCH","ENABLE_ROW_LATCH","ENABLE_SIM_OUTPUT_REGISTERS"])

HRR_mode_list.append(4)

verilog_parameters_list.append(["IN_WIDTH", "INPUT_REG_DEPTH", "MULT_PIPE_DEPTH"])

has_testbench_list.append(True)
testbench_functions_list.append(MatrixMatrixMultiply_SI_testbench_F)

In [131]:
# #MatrixMatrixMultiply_SoE

# functions_list.append(MatrixMatrixMultiply_SoE)
# functions_names_list.append("MatrixMatrixMultiply_SoE")

# parameters_list.append(["M","N","Q","PR","PC","NOEPS","MAMCS","ADDER_SIZE",
#                        "ENABLE_INPUT_LATCH","ENABLE_COLUMN_LATCH","ENABLE_ROW_LATCH","ENABLE_SIM_OUTPUT_REGISTERS"])

# HRR_mode_list.append(3)

# verilog_parameters_list.append(["IN_WIDTH", "INPUT_REG_DEPTH", "MULT_PIPE_DEPTH"])

# has_testbench_list.append(False)
# testbench_functions_list.append(None)

In [132]:
# #M_Type2

# functions_list.append(M_Type2)
# functions_names_list.append("M_Type2")

# HRR_mode_list.append(2)

In [133]:
# #SItoSoE

# functions_list.append(SItoSoE)
# functions_names_list.append("SItoSoE")

# parameters_list.append(["N","HRR","ENABLE_INPUT_LATCH","ENABLE_SIM_OUTPUT_REGISTERS"])

# HRR_mode_list.append(4)

# verilog_parameters_list.append(["IN_WIDTH"])

# has_testbench_list.append(False)
# testbench_functions_list.append(None)

Functions End

In [ ]:
verilog_parameters_ordered = ['IN_WIDTH', 'INPUT_REG_DEPTH', 'MULT_PIPE_DEPTH']

In [135]:
# print("choose function:")
# i=0;
# for n in functions_names_list:
#     i+=1
#     print("{}: ".format(i)+n)
# index = int(input("Enter function number([1,{}]):".format(i)))-1

In [136]:
index = 0

In [137]:
notation = { #will be updated after getting values
    "ENABLE_INPUT_LATCH": "[0(No),1(Yes)]",
    "ENABLE_COLUMN_LATCH": "[0(No),1(Yes)]",
    "ENABLE_ROW_LATCH": "[0(No),1(Yes)]",
    "ENABLE_SIM_OUTPUT_REGISTERS": "[0(No),1(Yes)]",
    #
    "M": ">=1",
    "Q": ">=1"
}

In [138]:
function = functions_list[index]
parameters = parameters_list[index]
HRRmode = HRR_mode_list[index]
if HRRmode in [1,2]:
    notation["N"] = ">=1"
elif HRRmode in [3,4]:
    notation["N"] = ">=2"

In [139]:
# parameter_values = {}
# notValid = False
# for parameter in parameters:
#     if parameter == "PR" and parameter_values["M"][1] == 1:
#         parameter_values["PR"] = [1,1]
#         RSmax = 1
#         print("PR: 1")
#         continue
#     if parameter == "PC" and parameter_values["Q"][1] == 1:
#         parameter_values["PC"] = [1,1]
#         CSmax = 1
#         print("PC: 1")
#         continue
#     if parameter == "NOEPS" and (
#     (HRmode == 2 and parameter_values["N"][1] == 1) or
#     (HRmode == 3 and parameter_values["N"][1] == 2) ):
#         parameter_values["NOEPS"] = [1,1]
#         print("NOEPS: 1")
#         continue
#     if parameter == "HRR" and parameter_values["N"][1] == 2:
#         parameter_values["HRR"] = [2,2]
#         print("HRR: 2")
#         continue
#     if parameter == "MAMCS" and (
#     (HRmode == 1 and parameter_values["N"][1] == 1) or
#     (HRmode == 4 and math.ceil(parameter_values["N"][1]/parameter_values["HRR"][0]) == 1) or
#     (HRmode in [2,3] and parameter_values["NOEPS"][1] == 1) ):
#         parameter_values["MAMCS"] = [1,1]
#         NOEPSmax = 1
#         print("MAMCS: 1")
#         continue
#     if parameter == "ADDER_SIZE":
#         if parameter_values["MAMCS"][0] == NOEPSmax:
#             parameter_values["ADDER_SIZE"] = [0,0]
#             print("ADDER_SIZE: 0")
#             continue
#         elif math.ceil(NOEPSmax/parameter_values["MAMCS"][0]) == 2: #else means parameter_values["MAMCS"][0] < NOEPSmax
#             parameter_values["ADDER_SIZE"] = [2,2]
#             print("ADDER_SIZE: 2")
#             continue
#     if parameter == "ENABLE_COLUMN_LATCH" and RSmax==1:
#         parameter_values["ENABLE_COLUMN_LATCH"] = [0,0]
#         print("ENABLE_COLUMN_LATCH: 0")
#         continue
#     if parameter == "ENABLE_ROW_LATCH" and CSmax==1:
#         parameter_values["ENABLE_ROW_LATCH"] = [0,0]
#         print("ENABLE_ROW_LATCH: 0")
#         continue

#     print(parameter+": "+notation[parameter])
#     startI = int(input(parameter+" start:"))
#     endI = int(input(parameter+" end:"))
#     if parameter == "N":
#         end = endI
#         if HRmode in [1,2]:
#             start = max(startI,1)
#             notation["NOEPS"] = "[1,{}]".format(end) #2
#         elif HRmode in [3,4]:
#             start = max(startI,2)
#             notation["HRR"] = "[2,{}] (not all HRR values are acceptable)".format(end) #4
#             notation["NOEPS"] = "[1,{}]".format(end-1) #3
#         if HRmode == 1:
#             NOEPSmax = end
#             notation["MAMCS"] = "[1,{}]".format(NOEPSmax)
#     elif parameter == "M":
#         start = max(startI,1)
#         end = endI
#         notation["PR"] = "[1,{}]".format(end)
#     elif parameter == "Q":
#         start = max(startI,1)
#         end = endI
#         notation["PC"] = "[1,{}]".format(end)
#     elif parameter == "PR":
#         start = max(startI,1)
#         end = min(endI,parameter_values["M"][1])
#         RSmax = math.ceil(parameter_values["M"][1]/start)
#     elif parameter == "PC":
#         start = max(startI,1)
#         end = min(endI,parameter_values["Q"][1])
#         CSmax = math.ceil(parameter_values["Q"][1]/start)
#     elif parameter in ["ENABLE_INPUT_LATCH","ENABLE_COLUMN_LATCH","ENABLE_ROW_LATCH","ENABLE_SIM_OUTPUT_REGISTERS"]: #Yes,No params
#         start = max(startI,0)
#         end = min(endI,1)
#     elif parameter == "HRR": #4
#         start = max(startI,2)
#         end = min(endI,parameter_values["N"][1])
#         NOEPSmax = math.ceil(parameter_values["N"][1]/start)
#         #the value itself might be invalid, but it's a correct upper limit
#         notation["MAMCS"] = "[1,{}]".format(NOEPSmax)
#     elif parameter == "NOEPS":
#         start = max(startI,1)
#         if HRmode == 2:
#             end = min(endI,parameter_values["N"][1])
#         elif HRmode == 3:
#             end = min(endI,parameter_values["N"][1]-1)
#         NOEPSmax = end
#         notation["MAMCS"] = "[1,{}]".format(NOEPSmax)
#     elif parameter == "MAMCS":
#         start = max(startI,1)
#         end = min(endI,NOEPSmax)
#         #if start < NOEPSmax:
#         notation["ADDER_SIZE"] = "[2,{}]".format(math.ceil(NOEPSmax/start)) #needed and correct for when (start < NOEPSmax)
#     elif parameter == "ADDER_SIZE":
#         start = max(startI,2)
#         end = min(endI,math.ceil(NOEPSmax/parameter_values["MAMCS"][0]))
#     else:
#         print("unknown parameter! will break.")
#         notValid = True
#         break;
#     print("calculated start: {}".format(start))
#     print("calculated end: {}".format(end))
#     if end<start:
#         print("Invalid value range! will break.")
#         notValid = True
#         break;
#     parameter_values[parameter]=[start,end]

In [140]:
parameter_values = {'M': [5, 5], 'N': [9, 9], 'Q': [5, 5], 'PR': [2, 2], 'PC': [2, 2], 'HRR': [4, 9],
'MAMCS': [1, 3], 'ADDER_SIZE': [2, 3], 'ENABLE_INPUT_LATCH': [1, 1],
'ENABLE_COLUMN_LATCH': [1, 1], 'ENABLE_ROW_LATCH': [1, 1], 'ENABLE_SIM_OUTPUT_REGISTERS': [1, 1]}

#temp changes to values
# parameter_values["NOEPS"][0] = 1 #2
# parameter_values["NOEPS"][1] = 4
# print(parameter_values)

print(parameter_values)

{'M': [5, 5], 'N': [9, 9], 'Q': [5, 5], 'PR': [2, 2], 'PC': [2, 2], 'HRR': [4, 9], 'MAMCS': [1, 3], 'ADDER_SIZE': [2, 3], 'ENABLE_INPUT_LATCH': [1, 1], 'ENABLE_COLUMN_LATCH': [1, 1], 'ENABLE_ROW_LATCH': [1, 1], 'ENABLE_SIM_OUTPUT_REGISTERS': [1, 1]}


In [141]:
INPUT_REG_DEPTH_max_valid = 2
MULT_PIPE_DEPTH_max_valid = 2

verilog_notation = {
    "IN_WIDTH": ">=1",
    "INPUT_REG_DEPTH": "[0,{}]".format(INPUT_REG_DEPTH_max_valid),
    "MULT_PIPE_DEPTH": "[0,{}]".format(MULT_PIPE_DEPTH_max_valid)
}

In [142]:
verilog_parameters = verilog_parameters_list[index]

In [143]:
# verilog_parameter_values = {}
# VnotValid = False
# for parameter in verilog_parameters:
#     print(parameter+": "+verilog_notation[parameter])
#     startI = int(input(parameter+" start:"))
#     endI = int(input(parameter+" end:"))
#     if parameter == "IN_WIDTH":
#         start = max(startI,1)
#         end = endI
#     elif parameter == "INPUT_REG_DEPTH":
#         start = max(startI,0)
#         end = min(endI,INPUT_REG_DEPTH_max_valid)
#     elif parameter == "MULT_PIPE_DEPTH":
#         start = max(startI,0)
#         end = min(endI,MULT_PIPE_DEPTH_max_valid)
#     else:
#         print("unknown parameter! will break.")
#         VnotValid = True
#         break;
#     print("calculated start: {}".format(start))
#     print("calculated end: {}".format(end))
#     if end<start:
#         print("Invalid value range! will break.")
#         VnotValid = True
#         break;
#     verilog_parameter_values[parameter]=[start,end]

In [144]:
verilog_parameter_values = {'IN_WIDTH': [10, 10], 'INPUT_REG_DEPTH': [1, 1], 'MULT_PIPE_DEPTH': [0, 1]}

#temp changes

print(verilog_parameter_values)

{'IN_WIDTH': [10, 10], 'INPUT_REG_DEPTH': [1, 1], 'MULT_PIPE_DEPTH': [0, 1]}


In [145]:
has_testbench = has_testbench_list[index]
testbench_function = testbench_functions_list[index]

In [146]:
print("no of parameters: {}".format(len(parameters)))
print()

family = "Spartan6"
device = "xc6slx150"
package = "fgg900"
speed_grade = "-3"

NoOfRresults = 0
NoOfParameters = len(parameters)
parameters_result = {}
ListOfAllParametersResults = []
#modules = {}

generate_params(0)

print("no of Results: {}".format(NoOfRresults))

# print(ListOfAllParametersResults)

NoOfVeilogParameters = len(verilog_parameters)
verilog_parameters_result = {}

no of parameters: 12

no of Results: 3


In [147]:
RandomIndices = range(NoOfRresults)
print(RandomIndices)

range(0, 3)


In [148]:
# #see NoOfRandomResults

# try:
#     del sys.modules['Generator_Random_List']
# except KeyError:
#     pass
# from Generator_Random_List import RandomIndices
# print(RandomIndices)

In [149]:
Print_To_File = 1

print_result_flag = True

make_base_files_flag = True

make_Synthesize_files_flag = False
Synthesize_flag = False

make_Simulate_files_flag = True
Simulate_flag = True

make_Simulate_files_flag = has_testbench and make_Simulate_files_flag

CPU_Cores_Count = 1
this_CPU = 0

Test_name = "TB"

Files_Location = "D:/Thesis_Results_"+Test_name+"_CPU{}/".format(this_CPU)
print("output directory: "+Files_Location)

NoOfCPUResults = 0
NoOfResultsT = 0

import datetime
print("start time: {}".format(datetime.datetime.now()))
print()

work_on_random_results_cpu()

print("no of CPU Results: {}".format(NoOfCPUResults))
print("total no of CPU verilog param modes: {}".format(NoOfResultsT))
print()
print("end time: {}".format(datetime.datetime.now()))

output directory: D:/Thesis_Results_TB_CPU0/
start time: 2018-01-17 20:30:22.924278

0:
{'M': 5, 'N': 9, 'Q': 5, 'PR': 2, 'PC': 2, 'HRR': 5, 'MAMCS': 1, 'ADDER_SIZE': 2, 'ENABLE_INPUT_LATCH': 1, 'ENABLE_COLUMN_LATCH': 1, 'ENABLE_ROW_LATCH': 1, 'ENABLE_SIM_OUTPUT_REGISTERS': 1}
{'SItoSoE_9_HRx5_IL_OR': 'SItoSoE', 'RegisteredMultiplier': 'RegisteredMultiplier', 'Registered2to1Adder_NIR': 'RegisteredNto1Adder', 'DotProduct_2_noHR_M_A2': 'DotProduct_noHR_M', 'DotProduct_9_S2E_HRx5_C1_A2': 'DotProduct_SoE', 'MatrixVectorMultiply_5_9_2PR_S2E_HRx5_C1_A2_VL': 'MatrixVectorMultiply_SoE', 'MatrixMatrixMultiply_5_9_5_2PR_2PC_S2E_HRx5_C1_A2_CL_RL': 'MatrixMatrixMultiply_SoE', 'MatrixMatrixMultiply_5_9_5_2PR_2PC_SI_HRx5_C1_A2_IL_CL_RL_MR': 'MatrixMatrixMultiply_SI'}
MatrixMatrixMultiply_5_9_5_2PR_2PC_SI_HRx5_C1_A2_IL_CL_RL_MR
no of verilog param modes = 2

1:
{'M': 5, 'N': 9, 'Q': 5, 'PR': 2, 'PC': 2, 'HRR': 5, 'MAMCS': 2, 'ADDER_SIZE': 0, 'ENABLE_INPUT_LATCH': 1, 'ENABLE_COLUMN_LATCH': 1, 'ENABLE_